# Лабораторная 2

In [1]:
from abc import ABCMeta, abstractmethod
import pandas as pd

In [2]:
df = pd.read_csv("data\creditcard.csv")

In [3]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [4]:
class Classificator():
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def train(X, Y):
        '''Обучить модель'''
    
    @abstractmethod
    def predict(self, X):
        '''Предсказать данные'''

In [5]:
class DecisionTree(Classificator):
    model = None
    
    def __init__():
        pass
    
    def train(X, Y, prediction, max_depth, leafValue):
        pass
    
    def predict(self, X):
        pass
    
    def entropy():
        pass
    
    def information_gain():
        pass

In [9]:
class RandomForest(Classificator):
    model = None
    
    def __init__():
        pass
    
    def train(X, Y):
        pass
    
    def predict(self, X):
        pass

In [10]:
class GradientBoosting(Classificator):
    model = None
    
    def __init__():
        pass
    
    def train(X, Y):
        pass
    
    def predict(self, X):
        pass